In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import copy

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from model import Symmetric, DeepSets, KNN, KK
from sample import generate_data, generate_narrow_data
from train import train
from evaluate import generalization_error, cross_validate

%matplotlib inline

In [ ]:
#For smooth neuron experiment, it's only fair to S2 if the neuron is drawn from the same random features

def smooth_neuron_weight_init(model, objective):
    if objective.__name__ == "smooth_neuron":
        if model.__name__ == "S2" or model.__name__ == "S3":
            with torch.no_grad():
                m = objective.__network__.phi.fc.weight.shape[0]
                model.phi.fc.weight[:m] = objective.__network__.phi.fc.weight
                model.phi.fc.weight.div_(torch.norm(model.phi.fc.weight, dim = 1, keepdim = True))

In [ ]:
def compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, objective, narrow, 
                   verbose = True, log_plot = False, scaleup = False, kernel_buff = False, squared = False):
    
    print("currently", objective.__name__)
    
    bias_first = "neuron" in objective.__name__

    c = 1 if not scaleup else 2
    
    k = 10 if kernel_buff else 1

    f1 = Symmetric(input_dim, c * hidden_dim, hidden_dim, squared = squared)
    f2 = KNN(input_dim, c * k * hidden_dim, hidden_dim, squared = squared)
    f3 = KK(input_dim, c * k * hidden_dim, k * hidden_dim, squared = squared)

    f1.__name__ = "S1"
    f2.__name__ = "S2"
    f3.__name__ = "S3"

    models = [f1, f2, f3]
    lambs = [0., 1e-6, 1e-4, 1e-2]
    N_list = np.arange(2, N_max + 16)

    for model in models:
        x, y = generate_data(N_max, batch_size, input_dim, objective, narrow, bias_first)
        
        smooth_neuron_weight_init(model, objective)
        
        cv_models = cross_validate(model, x, y, iterations, lambs, verbose)
        
        validation_errors = np.zeros_like(lambs)
        for i, cv_model in enumerate(cv_models):
            validation_errors[i] = generalization_error([N_max], 1000, input_dim, cv_model,
                                                        objective, narrow, bias_first)[0]
        
        i = np.argmin(validation_errors)
        lamb = lambs[i]
            
        runs = 10
        run_errors = np.zeros((runs, len(N_list)))
        for i in range(runs):
            x, y = generate_data(N_max, batch_size, input_dim, objective, narrow, bias_first)
            model_copy = copy.deepcopy(model)
            model_copy.reinit()
            smooth_neuron_weight_init(model_copy, objective)
            
            train(model_copy, x, y, iterations, lamb)
            errors = generalization_error(N_list, 1000, input_dim, model_copy, objective, narrow, bias_first)
            run_errors[i] = np.array(errors)
        
        mean_error = np.mean(run_errors, axis = 0)
        std_error = np.std(run_errors, axis = 0)
        if verbose:
            print("performance of ", model.__name__, " on ", objective.__name__)
            print("lamb =", lamb)
            print(mean_error)
            print(std_error)
            
            
        narrow_str = "Narrow" if narrow else "Wide"
        scaleup_str = "scaleup" if scaleup else ""
        save_str = model.__name__ + "_" + objective.__name__ + "_" + narrow_str + "_" + str(input_dim)
        save_str += "_" + str(scaleup) + "_" + str(kernel_buff) + "_" + str(squared)
        save_dir = "saved_data_2022/"
            
        np.save(save_dir + save_str + "_mean", mean_error)
        np.save(save_dir + save_str + "_std", std_error)
        
        if log_plot:
            plt.semilogy(N_list, mean_error, label = model.__name__)
        else:
            plt.plot(N_list, mean_error, label = model.__name__)
        plt.fill_between(N_list, mean_error - std_error, mean_error + std_error, alpha = 0.2)

    
    plt.legend()
    plt.ylim([1e-5, 1e2]) 

    plt.xlabel("N")
    plt.ylabel("Mean Square Error")
    narrow_str = "Narrow" if narrow else "Wide"
    plt.title(narrow_str + " generalization for " + objective.__name__)
    scaleup_str = "scaleup" if scaleup else ""
#     plt.savefig("plots_high_dim/" + objective.__name__ + "_" + narrow_str + "_" + str(input_dim) + scaleup_str)
    plt.show()
    plt.close()

In [ ]:
#For the rest of the notebook
input_dim = 10
hidden_dim = 100
squared = True

In [ ]:
mean = lambda x: np.mean(norm(x, axis = 2), axis = 1, keepdims = True)

median = lambda x: np.median(norm(x, axis = 2), axis = 1, keepdims = True)

maximum = lambda x: np.max(norm(x, axis = 2), axis = 1, keepdims = True)

lamb = 0.1
softmax = lambda x: lamb * np.log(np.mean(np.exp(norm(x, axis = 2) / lamb), axis = 1, keepdims = True))

second = lambda x: np.sort(norm(x, axis = 2), axis = 1)[:,-2].reshape(-1,1)

In [ ]:
mean_flip = lambda x: np.mean(1.0 / norm(x, axis = 2), axis = 1, keepdims = True)

median_flip = lambda x: np.median(1.0 / norm(x, axis = 2), axis = 1, keepdims = True)

maximum_flip = lambda x: np.max(1.0 / norm(x, axis = 2), axis = 1, keepdims = True)

lamb = 0.1
softmax_flip = lambda x: lamb * np.log(np.mean(np.exp(1.0 / norm(x, axis = 2) / lamb), axis = 1, keepdims = True))

second_flip = lambda x: np.sort(1.0 / norm(x, axis = 2), axis = 1)[:,-2].reshape(-1,1)

In [ ]:
from scipy.spatial import distance_matrix

def potential(x):
    energies = np.zeros((x.shape[0], 1))
    for i in range(x.shape[0]):

        r = x[i]
        D = distance_matrix(r, r)

        np.fill_diagonal(D, 1)
        D = 1.0/D
        
        m = D.shape[0]
        r,c = np.triu_indices(m,1)
        D = D[r,c]
        energies[i] = -np.mean(D)
        
    return energies

In [ ]:
def mixture(tensor, mean_1, std_1, mean_2, std_2):
    with torch.no_grad():
        x_1 = mean_1 + std_1 * torch.randn_like(tensor)
        x_2 = mean_2 + std_2 * torch.randn_like(tensor)
        
        p = torch.bernoulli(torch.zeros_like(tensor) + 0.5)
        tensor.data = p * x_1 + (1-p) * x_2


In [ ]:
### May need to sample several neurons to find one that isn't degenerate on the domain

torch.manual_seed(50)
np.random.seed(50)

for i in range(100):
    teacher = Symmetric(input_dim, 1, 1, squared = squared)
    mixture(teacher.phi.fc.weight, 1.0, 0.5, -1.0, 0.5)
    teacher.eval()

    def neuron(x):
        x = torch.from_numpy(x).float()
        y = teacher(x)
        return y.data.numpy().reshape(-1, 1)

    neuron.__network__ = teacher

    x, y = generate_narrow_data(3, 15, input_dim, neuron, bias_first = True)
    z = y.data.numpy().flatten()
    print(z)
    if np.count_nonzero(z==0) >= 1 and np.count_nonzero(z==0) < 4 and np.max(np.abs(z)) >= 3:
        break

In [ ]:
### May need to sample several neurons to find one that isn't degenerate on the domain

smooth_teacher = Symmetric(input_dim, hidden_dim, 1, squared = squared)
# mixture(smooth_teacher.rho.fc1.weight, 0.5, 0.5, -0.5, 0.5)
smooth_teacher.eval()
def smooth_neuron(x):
        x = torch.from_numpy(x).float()
        y = smooth_teacher(x)
        return y.data.numpy().reshape(-1, 1)

smooth_neuron.__network__ = smooth_teacher

x, y = generate_narrow_data(3, 15, input_dim, smooth_neuron, bias_first = True)
print(y.data.numpy().flatten())

In [ ]:
neuron.__name__ = "neuron"
smooth_neuron.__name__ = "smooth_neuron"
maximum.__name__ = "maximum"
softmax.__name__ = "softmax"
median.__name__ = "median"
mean.__name__ = "mean"
second.__name__ = "second"
potential.__name__ = "potential"


maximum_flip.__name__ = "maximum_flip"
softmax_flip.__name__ = "softmax_flip"
median_flip.__name__ = "median_flip"
mean_flip.__name__ = "mean_flip"
second_flip.__name__ = "second_flip"

In [ ]:
###############################################

In [ ]:
#Run to generate plots in Figure 1:

N_max = 4

iterations = 5000
batch_size = 100

In [ ]:
compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, smooth_neuron, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)
compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, neuron, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)

compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, smooth_neuron, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared, scaleup = True)
compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, neuron, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared, scaleup = True)

In [ ]:
compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, potential, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)

In [ ]:
compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, mean_flip, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)

compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, median_flip, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)

compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, maximum_flip, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)

compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, softmax_flip, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)

compare_models(N_max, hidden_dim, iterations, batch_size, input_dim, second_flip, narrow = False, log_plot = True,
              kernel_buff = True, squared = squared)


